In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from urlAnalysis import create_graph

In [ ]:
G = create_graph()

In [ ]:
from utils import analyze_url
pubs = []
for r in G.predecessors('http://sci-lens.org#repository'):
    for n in G.predecessors(r):
        domains = set()
        for w in G.predecessors(n):
            domain, _ = analyze_url(w)
            domains.add(domain)
        pubs.append([n, len(domains)])
pubs = pd.DataFrame(pubs)
pubs = pubs.sort_values(1, ascending=False)

In [ ]:
pubs[pubs[1]>5][0].to_csv('out.txt', index=False)

In [ ]:
df = pd.read_csv(diffusion_graph_dir+'epoch_0.tsv', sep='\t').dropna()
df = df.drop(['target_url'], axis=1)
df['social'] = 'http://sci-lens.org#twitter'

In [ ]:
G =  nx.compose(G, nx.from_pandas_edgelist(df, source='social', target='source_url', create_using=nx.DiGraph()))

In [ ]:
for index, row in df.iterrows():
    G.add_node(row['source_url'], popularity=row['popularity'], timestamp=row['timestamp'], user_country=row['user_country'])

In [ ]:
pubs = []
for r in G.predecessors('http://sci-lens.org#repository'):
    for n in G.predecessors(r):
        for m in G.predecessors(n):
            print (m)
#         popularity = 0
#         for path in nx.all_simple_paths(G, source='http://sci-lens.org#twitter', target=n):
#             popularity += G.node[path[1]]['popularity']
#         pubs.append([n , popularity])



In [ ]:
for n in G.predecessors('http://www.cdc.gov/mmwr/volumes/65/ss/ss6512a1.htm'):
    print(n)

In [ ]:
TopNodes = sorted(G.in_degree, key=lambda x: x[1], reverse=True)

In [ ]:
for t in TopNodes[:40]:
    print(t[1], '\t', t[0])